# Exercises Week 11-12: Graded exercise session (part B)

**Course**: [Life Sciences engineering: genome to function](https://go.epfl.ch/BIO-411) (BIO-411)

**Professors**:  _Gönczy Pierre_, _Naef Felix_, _McCabe Brian Donal_

SSV, MA, 2023-2024

In [3]:
import pandas as pd
import numpy as np
from sklearn.decomposition import PCA
import plotly.express as px
import matplotlib.pyplot as plt
from sklearn.preprocessing import scale

from scipy.integrate import odeint
import matplotlib.pyplot as plt
from ipywidgets import interact, fixed
from scipy.stats import beta
import math

**Provide answers in the form of code, figures (if relevant) and short descriptions (in markdown cells) in those notebooks. Submit your notebook to Moodle, please make sure to execute every cell.**

### Exercise 1: Simulation of circadian gene regulation

In this exercise, we revisit and extend the concepts from Week 9, focusing on the transcription and splicing of a pre-mRNA to produce mRNA. We describe this process using a two-dimensional Ordinary Differential Equation (ODE), similar to our previous approach but with a key difference: we now incorporate **time-dependent** transcription and degradation rates to model circadian rhythms.

The system is defined by the following ODEs:

\begin{array}{ll}
\frac{dP}{dt} = s(t) - \rho P \\  
\\
\frac{dM}{dt} = \rho P - k(t) M \\
\end{array}

where  $P$ and $M$ represent the concentrations of pre-mRNA and mRNA, respectively. We will consider time-dependent (circadian) transcription and degradation rates. The frequency is defined by $\omega=\frac{2\pi}{T}$ with $T=24h$.  

$s(t)$ is the transcription rate, which is now taken either as constant $s(t)=s_{0}$ or as a rhythmic function of time:  

\begin{equation*}
s(t) = s_0(1 + \epsilon_s \cos(\omega t)) 
\end{equation*}  
with relative amplitude $0\leq \epsilon_s\leq 1$. Note that the peak of $s(t)$ is at $t=0$.  
 
Similarly, we will consider the degradation rate $k(t)$ either as constant $k(t)= k_0$ or a rhythmic function of time:  
\begin{equation*}
k(t)=k_0(1 + \epsilon_k \cos(\omega (t-t_k))
\end{equation*}  
with $0\leq\epsilon_k\leq 1$, and the maximum degradation rate at $t=t_k$.

$\rho$ represents the constant splicing rate of pre-mRNA $P$ into mRNA $M$.

### Question 1
1. Adapt the code from Week 9 exercises to simulate this system with rhythmic transcription and degradation rates. Modify the interactive widget such that you can vary the new parameters and plot $P(t)$ and $M(t)$.  

2. Keep the degradation rate constant (set $\epsilon_k$=0) and vary $k_0$ and $\epsilon_s$.
    * How does the phase delay between $M(t)$ and $P(t)$ changes with different values of $k_0$?
    * How does the amplitude (use the ratio between max and min) of $M(t)$ changes with different values of $k_0$? 
    
    Make sure you span a relevant range for $k_0^{-1}$ ( *i.e.* from 10 minutes to several hours).  

3. Now introduce rhythmic degradation by varying $\epsilon_k$. 
    * Show that you can obtain a rhyhtmic mRNA accumulatiokn when the synthesis is constant ($\epsilon_s=0$). 
    * Explore and discuss scenarios where the introduction of rhythmic degradation leads to dynamics that cannot be achieved with a constant half-life.  
    
    See also [Wang et al., 2018](https://www.pnas.org/doi/10.1073/pnas.1715225115).

In [ ]:
def production_decay_ode(y, time, rho, s0, es, k0, ek, tk ):

    """
    Defines the ODEs for the production and decay of pre-mRNA and mRNA.
    P: pre-mRNA concentration
    M: mRNA concentration
    st: transcription rate (time dependent)
    kt: mRNA degradation rate (time dependent)
    rho: splicing rate
    s0: transcription constant
    es: relative amplitude of the transcription rate oscillation
    ek: relative amplitude of the mRNA degradation rate oscillation
    k0: degradation constant
    time: time point (in minutes)
    tk: max degrade rate

    #kp: pre-mRNA degradation rate
    #km: mRNA degradation rate
    #tau: protein synthesis rate
    #kg: protein degradation rate
    """
    omega = 2*np.pi/(24*60)
    P, M = y
    st = s0*(1+es*math.cos(omega*time))
    kt = k0*(1+ek*math.cos(omega*(time-tk)))
    dPdt = st - rho*P
    dMdt = rho*P - kt*M
    return [dPdt, dMdt] 

def simulation(ode_function, parameters, initial_conditions = None, time = None):

    """
    Simulate the ODEs for the production and decay of pre-mRNA and mRNA.
    parameters: parameters for the ODEs
    initial_conditions: initial conditions for the ODEs
    time: time intervals for the simulation
   
    s = parameters['s']
    kp = parameters['kp']
    km = parameters['km']
    rho = parameters['rho']
    tau = parameters['tau']
    kg = parameters['kg']
    """
    
    
    
    rho= parameters['rho']
    s0= parameters['s0']
    es= parameters['es']
    k0= parameters['k0']
    ek= parameters['ek']
    #time= parameters['time']
    tk= parameters['tk']
    
    #omega = 2*np.pi/(24*60)
    #st = s0*(1+es*math.cos(omega*time))
    #kt = k0*(1+ek*math.cos(omega*(time-tk)))
    # Placeholder for adding steady-state solutions

    P_ss = s0 / rho #Add your steady-state solution for P here 
    M_ss = s0 / k0 #Add you steady-state solution for M here
    
    #results = odeint(ode_function, initial_conditions, time, args=(rho, s0, es, k0, ek, tk))
    results = odeint(ode_function, initial_conditions, time, args=(rho, s0, es, k0, ek, tk))

    plt.figure(figsize=(10, 6))
    plt.ylim([0,150])

    # Uncomment the lines below after adding your steady-state solutions
    plt.axhline(y=P_ss, color='blue', linestyle='--', label='P_ss')  
    plt.axhline(y=M_ss, color='orange', linestyle='--', label='M_ss')

    plt.title('Simulation of pre-mRNA and mRNA Production-Decay Dynamics')
    plt.plot(time, results[:, 0], label = 'P(t)', linewidth=2)
    plt.plot(time, results[:, 1], label = 'M(t)', linewidth=2)
    plt.legend()

    plt.xlabel("Time [min]")
    plt.ylabel("Concentration")
    plt.legend(loc='upper left')
    plt.show()
    
def animate(initial_conditions, time, rho=0.4, s0=5, es=1, k0=0.05, ek=1, tk=0.5 ):
    """
    Runs the simulation with an interactive widgets for parameter exploration.
    """
    #args=(rho, s0, es, k0, ek, time, tk )
    parameters = {'rho':rho, 's0':s0, 'es':es, 'k0':k0, 'ek':ek, 'tk':tk}
    simulation(production_decay_ode, parameters, initial_conditions, time)


In [ ]:
# Define initial conditions and time intervals for the simulation
initial_conditions = [0, 0] #[P, M, G]
time = np.linspace(0, 500, 10001)

interact(animate, 
         initial_conditions=fixed(initial_conditions), 
         time=fixed(time), 
        rho=(0, 1, 0.1),
        s0=(0, 10, 0.1),
        es=(0, 1, 0.1),
        k0=(1/600,1/10,1/1000),
        ek=(0, 1, 0.1),
        tk=(0, 1, 0.1));

interactive(children=(FloatSlider(value=0.4, description='rho', max=1.0), FloatSlider(value=5.0, description='…

### Exercise 2: Circadian (post-)transcriptional regulation of gene expression in mouse liver

In this exercise, we will analyse published RNA-seq data from mouse liver under an *ad libitum* feeding regimen and in presence of a 12-hour light-dark cycle ([Atger et al., 2015](https://www.pnas.org/doi/abs/10.1073/pnas.1515308112)). The samples were harvested every 2 hours in four replicates, RNA was extracted and sequenced. Similarly to Week 9 exercises on RNA-seq data, gene expression was quantified at the intron and exon levels. Note that data are $log_2$ transformed and normalized (RPKM).

### Question 1
1. Modify the code from Week 9 exercises to perform PCA using the 14 following genes (core clock genes and selected clock output genes):  
 ``['Arntl', 'Npas2', 'Clock', 'Cry1', 'Cry2', 'Per1', 'Per2', 'Per3', 'Nr1d1', 'Nr1d2', 'Rora', 'Rorc', 'Tef', 'Dbp']``.  
 First start with only the exons, then only the introns, and eventually both. Describe your observations.  

2. Use the *return_amp_phase_pv* function on the $log_2$ data to assess and characterize rhythmicity genome-wide.
    * Select genes based on p-value ($p<0.01) and amplitude (log2 fold-change>0.5) thresholds for further analysis.
    * Provide histograms of the peak times, and amplitudes ($log_2$ peak-to-trough) for the selected genes.
    * Perform this analysis separately for the exons and introns.
    * Examine whether you observe patterns typically referred to as the 'morning' and 'evening' waves of gene expression.
    
3. Determine which genes show the highest amplitude rhythmic patterns. Are there any common themes or characteristics among these genes?

#### Load the RNA-seq data

In [ ]:
dat = pd.read_csv("./GSE73554_WT_AL_Intron_Exon_RFP.txt",sep='\t')
dat.index = dat['Gene_Symbol']
dat = dat.drop(['Gene_Symbol','Gene_Ensembl'], axis=1)
dat.columns = dat.columns.str.split('_', expand = True)
dat.columns.names = ['condition','feeding','feature','time','replicate']

dat.head()

condition          WT                                                    \
feeding            AL                                                     
feature        Intron                                                     
time               00        02        04        06        08        10   
replicate           A         A         A         A         A         A   
Gene_Symbol                                                               
Gnai3       -1.564402 -1.536626 -1.931707 -1.458354 -1.575970 -1.654038   
Cdc45       -3.838233 -4.132913 -4.023130 -3.782926 -3.971205 -3.898270   
Apoh        -1.075680 -1.540220 -1.569142 -1.652602 -1.786800 -1.539642   
Narf        -1.198793 -0.937221 -0.921084 -0.331866 -0.123671 -0.186613   
Cav2        -3.046334 -3.741371 -3.364087 -3.063080 -3.374724 -2.992739   

condition                                            ...             \
feeding                                              ...              
feature                                              ...        RFP   
time               12        14        16        18  ...         04   
replicate           A         A         A         A  ...          D   
Gene_Symbol                                          ...              
Gnai3       -1.550707 -1.564596 -1.211372 -1.654187  ...   4.739938   
Cdc45       -3.837377 -3.907227 -4.002231 -3.837719  ...  -1.783427   
Apoh        -1.434320 -1.655522 -1.499171 -1.411709  ...  10.657651   
Narf        -0.632810 -0.963087 -0.999763 -1.320254  ...   1.788997   
Cav2        -2.985271 -2.991166 -2.954387 -3.052015  ...   0.759856   

condition                                                                      \
feeding                                                                         
feature                                                                         
time                06         08         10         12         14         16   
replicate            D          D          D          D          D          D   
Gene_Symbol                                                                     
Gnai3         4.648414   4.576937   4.891934   5.284247   5.502545   5.115915   
Cdc45        -1.045727  -1.868940  -1.616297   1.552717   0.336330  -1.370050   
Apoh         10.460193  10.623698  10.381588  10.146668  10.232355  10.631922   
Narf          2.660656   2.890531   3.379632   2.924193   2.472548   2.686118   
Cav2          0.403964   0.581452   0.716145   0.742948   0.385111   0.282426   

condition                                     
feeding                                       
feature                                       
time                18         20         22  
replicate            D          D          D  
Gene_Symbol                                   
Gnai3         5.402904   5.119890   4.873039  
Cdc45        -0.308212  -1.218384  -0.541827  
Apoh         10.468625  10.704931  10.591210  
Narf          2.390108   1.941836   2.128767  
Cav2          0.624375   0.806800   0.524357  

[5 rows x 144 columns]

In [ ]:
def subselect(data, feature, replicate, time, condition, feeding):

    """
    Subselects samples from the full dataset based on the specified features, conditions, 
    replicates, and time points.

    Parameters:
    data: The full gene expression dataset.
    feature: The features to filter by (e.g., ['intron', 'exon']).
    replicate: The replicates to include.
    time: The time points to include.
    condition: The temperature shift conditions to include (e.g, ['33to38', '38to33']).

    Returns:
    pd.DataFrame: The filtered dataset.
    """
    
    # Create boolean masks for each filtering criterion

    feature_mask  = data.columns.get_level_values('feature').isin(feature)
    condition_mask  = data.columns.get_level_values('condition').isin(condition)
    replicate_mask  = data.columns.get_level_values('replicate').isin(replicate)
    time_mask = data.columns.get_level_values('time').isin(time)
    feeding_mask = data.columns.get_level_values('feeding').isin(feeding)
    # Use the masks to filter the data

    #filtered_data = data.iloc[:, feature_mask & condition_mask & replicate_mask & time_mask]
    filtered_data = data.iloc[:, feature_mask & condition_mask & replicate_mask & time_mask & feeding_mask]
    return filtered_data

def run_PCA(data, n_components=5, color_by='replicate', symbol_by='time', scale_data=True, log_transform=True):
    
    """
    Runs PCA on the provided data and generates a scatter plot of pairs of principal components.
    It also returns a DataFrame containing the PCA loadings.

    Parameters:
    data: The gene expression data to analyze.
    n_components: The number of principal components to calculate.
    color_by: The metadata category to color the data points by.
    symbol_by: The metadata category to symbolize the data points by.
    scale_data: Whether to scale the data before PCA.
    log_transform: Whether to apply log transformation to the data.

    Returns:
    pd.DataFrame: A DataFrame containing the PCA loadings.
    """

    data_tmp = data.copy()
    
    # Scale and/or log-transform the data if specified
    if scale_data:
        data = scale(data)
    else:
        data = data
        
    if log_transform:
        data = np.log2(data + 1)
        
    # Fit the PCA model
    pca = PCA(n_components=n_components)
    pca.fit(data.T)

    # Transform the data
    pca_transformed = pca.transform(data.T)

    # Create a DataFrame for the PCA results
    labels = [f'PC{i+1}' for i in range(n_components)]
    pca_df = pd.DataFrame(pca_transformed, columns=labels)
    pca_df[color_by] = data_tmp.columns.get_level_values(color_by)
    pca_df[symbol_by] = data_tmp.columns.get_level_values(symbol_by)

    # Plotting each pair of consecutive principal components
    fig, axes = plt.subplots(1, n_components -1, figsize=(30, 5))
    for i in range(n_components-1):
        sns.scatterplot(ax=axes[i], data=pca_df, x=f'PC{i+1}', y=f'PC{i+2}', 
        hue=color_by, style=symbol_by, s=200, palette='Set1')

        axes[i].set_xlabel(f'PC{i+1} ({pca.explained_variance_ratio_[i]*100:.2f}%)')
        axes[i].set_ylabel(f'PC{i+2} ({pca.explained_variance_ratio_[i+1]*100:.2f}%)')
        # remove legend if it's not the first plot
        if i != 0:
            axes[i].get_legend().remove()
    
    fig.suptitle(f'PCA of Gene Expression Data (color={color_by}, symbol={symbol_by})', fontsize=16)

    # Return the PCA loadings
    df_pca = pd.DataFrame(pca.components_, columns=data_tmp.index, index=labels)

    return df_pca


## Plot Gene profile across all conditions
def plot_gene(data, gene , xx_33_38=None, xx_38_33=None, t_2=None):

    """
    Plots the gene expression profile across different conditions and time points.
    
    Parameters:
    data: The gene expression data.
    gene: The gene to plot.
    xx_33_38: The simulated solution for the 33to38 condition.
    xx_38_33: The simulated solution for the 38to33 condition.

    Returns:
    None
    """

    #subselect
    time= ['0', '1' ,'3' ,'6', '9']
    replicate = ['2', '13', '11', '5']
    dat_1= subselect(data, ['intron'], replicate, time, ['33to38'])
    dat_2= subselect(data, ['exon'], replicate, time, ['33to38'])
    dat_3= subselect(data, ['intron'], replicate, time, ['38to33'])
    dat_4= subselect(data, ['exon'], replicate, time, ['38to33'])

    #plot
    t=np.array([0, 0, 1, 1, 3, 3, 6, 6, 9 , 9])
    fig, axs = plt.subplots(2, 2, figsize=(8,8), sharex=True)
    
    axs[0, 0].plot(t, dat_1[dat_1.index.str.endswith(gene)].values.T, 'o', color = 'lightblue', label='intron 33to38')
    axs[0, 0].set_title("intron 33to38")
    
    axs[1, 0].plot(t, dat_2[dat_2.index.str.endswith(gene)].values.T, 'o', color = 'lightblue', label='exon 33to38')
    axs[1, 0].set_title("exons 33to38")
    
    axs[0, 1].plot(t, dat_3[dat_3.index.str.endswith(gene)].values.T, 'o', color = 'lightblue', label='intron 38to33')
    axs[0, 1].set_title("intron 38to33")

    axs[1, 1].plot(t, dat_4[dat_4.index.str.endswith(gene)].values.T, 'o', color = 'lightblue', label='exon 38to33')
    axs[1, 1].set_title("exon 38to33")

    if xx_33_38 is not None:
        axs[0,0].plot(t_2, xx_33_38[:,0],label = 'P(t)')
        axs[1,0].plot(t_2, xx_33_38[:,1])
        axs[0,1].plot(t_2, xx_38_33[:,0])
        axs[1,1].plot(t_2, xx_38_33[:,1])
    
        
    for ax in axs.flat:
        ax.set(xlabel='Time [h]', ylabel='RPKM')

    fig.suptitle(f'Gene Expression Profile for {gene}', fontsize=16)
    plt.show()

def plot_DE(dat_all, condition_x1, condition_x2, condition_y1, condition_y2, time_x1, time_x2, time_y1, time_y2, feature_x1, feature_x2, feature_y1, feature_y2, FC=2, xlab="", ylab="", FC_type="delta"):
    
    """
    Plots differentially expressed genes based on log fold changes between specified conditions, times, and features.

    Parameters:
    dat_all: The complete dataset with gene expression values.
    condition_x1, condition_x2, condition_y1, condition_y2: Lists of conditions to compare.
    time_x1, time_x2, time_y1, time_y2: Lists of time points to compare.
    feature_x1, feature_x2, feature_y1, feature_y2: Lists of features to compare.
    FC: Absolute fold change threshold for considering a gene differentially expressed in at least one of the two axes.
    xlab: Label for the x-axis representing the log fold change between condition_x2 and condition_x1.
    ylab: Label for the y-axis representing the log fold change between condition_y2 and condition_y1.

    Returns:
    Array: Boolean array indicating differentially expressed genes that meet the fold change threshold.
    """
     
    replicate = ['2', '13', '11', '5']

    dat_x1=subselect(dat_all, feature_x1, replicate, time_x1, condition_x1).mean(axis=1)
    dat_x2=subselect(dat_all, feature_x2, replicate, time_x2, condition_x2).mean(axis=1)

    dat_y1=subselect(dat_all, feature_y1, replicate, time_y1, condition_y1).mean(axis=1)
    dat_y2=subselect(dat_all, feature_y2, replicate, time_y2, condition_y2).mean(axis=1)

    x=np.log2(1 + dat_x2)- np.log2(1 + dat_x1)
    y=np.log2(1 + dat_y2) - np.log2(1 + dat_y1)

    if FC_type == "delta":
        pos =((x > FC) & (y > FC)) | ((x < -FC) & (y < -FC))

    if FC_type == "delta_delta":
        pos = abs(x-y) > FC

    x_2 = x[pos]
    y_2 = y[pos]
    fig, ax = plt.subplots(figsize=(10,10))

    ax.plot(x,y, 'o', color = 'lightblue', markersize=2)
    ax.axhline(y=0, color='b')
    ax.axvline(x=0, color='b')
    ax.set_xlim(-8,8)
    ax.set_ylim(-8,8)
    ax.set_xlabel(xlab)
    ax.set_ylabel(ylab)
    texts = [ax.text(x_2[k], y_2[k], v.split("|")[1]) for k, v in enumerate(dat_all.index[pos])]
    adjust_text(texts)
    
    return(pos)    


### Question 1
1. Modify the code from Week 9 exercises to perform PCA using the 14 following genes (core clock genes and selected clock output genes):  
 ``['Arntl', 'Npas2', 'Clock', 'Cry1', 'Cry2', 'Per1', 'Per2', 'Per3', 'Nr1d1', 'Nr1d2', 'Rora', 'Rorc', 'Tef', 'Dbp']``.  
 First start with only the exons, then only the introns, and eventually both. Describe your observations.  

2. Use the *return_amp_phase_pv* function on the $log_2$ data to assess and characterize rhythmicity genome-wide.
    * Select genes based on p-value ($p<0.01) and amplitude (log2 fold-change>0.5) thresholds for further analysis.
    * Provide histograms of the peak times, and amplitudes ($log_2$ peak-to-trough) for the selected genes.
    * Perform this analysis separately for the exons and introns.
    * Examine whether you observe patterns typically referred to as the 'morning' and 'evening' waves of gene expression.
    
3. Determine which genes show the highest amplitude rhythmic patterns. Are there any common themes or characteristics among these genes?

In [ ]:
dat = pd.read_csv("./GSE73554_WT_AL_Intron_Exon_RFP.txt",sep='\t')
dat.index = dat['Gene_Symbol']
dat = dat.drop(['Gene_Symbol','Gene_Ensembl'], axis=1)
dat.columns = dat.columns.str.split('_', expand = True)
dat.columns.names = ['condition','feeding','feature','time','replicate']
list_of_genes = ['Arntl', 'Npas2', 'Clock', 'Cry1', 'Cry2', 'Per1', 'Per2', 'Per3', 'Nr1d1', 'Nr1d2', 'Rora', 'Rorc', 'Tef', 'Dbp']
# show the different variables in condition of dat_genes
dat_genes.columns.get_level_values('time').unique()

#extract from dat the genes in list_of_genes  
dat_genes = dat.loc[list_of_genes]
dat_genes.head()


condition          WT                                                    \
feeding            AL                                                     
feature        Intron                                                     
time               00        02        04        06        08        10   
replicate           A         A         A         A         A         A   
Gene_Symbol                                                               
Arntl       -1.669041 -1.694083 -2.706959 -3.508204 -3.848818 -4.003520   
Npas2       -1.962732 -2.192843 -3.944118 -4.209720 -4.256702 -4.273067   
Clock       -1.597633 -1.549136 -1.794397 -1.740208 -2.561197 -2.653104   
Cry1        -3.015745 -2.888921 -3.226192 -3.355850 -3.444067 -2.907677   
Cry2        -2.125658 -1.418011 -1.267150 -0.570111 -0.933872 -0.568488   

condition                                            ...                      \
feeding                                              ...                       
feature                                              ...       RFP             
time               12        14        16        18  ...        04        06   
replicate           A         A         A         A  ...         D         D   
Gene_Symbol                                          ...                       
Arntl       -3.509194 -2.494314 -2.238684 -2.282043  ...  3.022983  1.673178   
Npas2       -4.269427 -4.246326 -4.237317 -4.149953  ... -0.111984 -3.286106   
Clock       -2.276997 -2.292100 -1.115022 -1.490327  ...  1.518437  1.848053   
Cry1        -2.312536 -1.554579 -1.367280 -1.529984  ...  1.572807  1.120746   
Cry2        -0.830493 -0.808692 -0.764942 -0.753802  ...  2.403922  3.050125   

condition                                                                \
feeding                                                                   
feature                                                                   
time               08        10        12        14        16        18   
replicate           D         D         D         D         D         D   
Gene_Symbol                                                               
Arntl       -0.323042 -1.531355  0.016196  1.502431  0.612706  3.621233   
Npas2       -3.129760 -3.995698 -4.009316 -4.020822 -3.073643 -0.001143   
Clock        1.144557  0.770371  1.090088  1.398042  1.619607  2.039359   
Cry1         0.413170  1.039117  1.978841  2.477187  0.829508  2.543046   
Cry2         3.369495  3.397454  3.530218  3.475627  3.426552  2.585690   

condition                        
feeding                          
feature                          
time               20        22  
replicate           D         D  
Gene_Symbol                      
Arntl        2.906462  3.175841  
Npas2       -2.603965 -0.873309  
Clock        1.611866  1.753404  
Cry1         3.048787  3.098065  
Cry2         3.121378  3.092411  

[5 rows x 144 columns]

In [ ]:
time= ['00', '02', '04', '06', '08', '10', '12', '14', '16', '18', '20', '22']
replicate = ['A', 'B', 'C', 'D']
feeding = ['AL']
conditions = ['WT']
feature = ['Intron', 'Exon']


dat_intron = subselect(dat_genes, ['Intron'], replicate, time, conditions, feeding)
dat_exon = subselect(dat_genes, ['Exon'], replicate, time, conditions, feeding)
dat_both = subselect(dat_genes, feature, replicate, time, conditions, feeding)


In [ ]:
# Run PCA on the gene expression data
time= ['00', '02', '04', '06', '08', '10', '12', '14', '16', '18', '20', '22']
replicate = ['A', 'B', 'C', 'D']
feeding = ['AL']
conditions = ['WT']
feature = ['Intron', 'Exon']
#Subselect the samples 
dat_sub = subselect(dat_both, feature, replicate, time, conditions, feeding)

df_pca = run_PCA(dat_both, n_components=2, color_by='feature', symbol_by='replicate', scale_data=True, log_transform=True)
#df_pca = run_PCA(dat_genes, n_components=2, color_by='exon', symbol_by='time', scale_data=True, log_transform=True)

C:\Users\morri\AppData\Local\Temp\ipykernel_9640\2838016254.py:58: RuntimeWarning: invalid value encountered in log2
  data = np.log2(data + 1)


ValueError: Input X contains NaN.
PCA does not accept missing values encoded as NaN natively. For supervised learning, you might want to consider sklearn.ensemble.HistGradientBoostingClassifier and Regressor which accept missing values encoded as NaNs natively. Alternatively, it is possible to preprocess the data, for instance by using an imputer transformer in a pipeline or drop samples with missing values. See https://scikit-learn.org/stable/modules/impute.html You can find a list of all estimators that handle NaN values at the following page: https://scikit-learn.org/stable/modules/impute.html#estimators-that-handle-nan-values

#### Assess rhythmicity

In [ ]:
# Function to infer the p-value, phase, amplitude and mean from a time-serie y with size N, period T and sampling Ts. 
def return_amp_phase_pv(y, Ts, T, N):
    
    """
    Perform harmonic regression using a Fourier series to infer the amplitude, phase, p-value, and mean from a time series.

    Parameters:
    y: Time series data.
    Ts: Sampling interval.
    T: Period of the time series.
    N: Number of data points in the time series.

    Returns:
    tuple: A tuple containing amplitude, phase, p-value, and mean of the time series.

    Notes:
    - The function uses Fast Fourier Transform (FFT) to decompose the time series.
    - Harmonic regression is performed to fit a cosine wave to the data.
    - The p-value is calculated based on the fit statistics, providing an estimate of the fit quality.
    """
      
    # Time vector for the series
    t = np.linspace(0.0, N * Ts, N)

    # Fourier Transform
    x_fft = np.fft.fft(y)
    freq = np.fft.fftfreq(len(y), d=Ts)

    # Identify the frequency closest to 1/T
    index, = np.where(np.isclose(freq, 1/T, atol=0.005))

    # Calculate amplitude and phase
    amp = 4 / N * np.abs(x_fft[index[0]])
    phase = T * np.arctan2(-x_fft[index[0]].imag, x_fft[index[0]].real) / (2 * np.pi)

    # Mean of the time series
    mu = 1 / N * x_fft[0].real
    
    # Harmonic regression to compute residuals and statistics
    x_hat = mu + 0.5 * amp * np.cos(2 * np.pi / T * t - 2 * np.pi * phase / T)
    res = y - x_hat
    sig2_1 = np.var(res)
    sig2 = np.var(y)
    R2 = 1 - sig2_1 / sig2

    # Compute p-value
    p = 3
    pval = 1 - beta.cdf(R2, (p - 1) / 2, (N - p) / 2)

    # Adjust phase to be within [0, T]
    if phase < 0:
        phase += T
        
    return amp, phase, pval, mu

In [ ]:
def process_data(data, feature_type, Ts, T, N):
    """
    Applies the return_amp_phase_pv function to all genes for a given feature type (intron or exon).

    Parameters:
    data: The dataset containing gene expression data.
    feature_type: Type of feature to process ('Intron' or 'Exon').
    Ts: Sampling time.
    T: Period.
    N: Number of samples.

    Returns:
    dict: A dictionary containing arrays for amplitude, phase, p-value, and mean for the specified feature type.
    """

    feature_pos = data.columns.get_level_values('feature').isin([feature_type])
    results = {'amp': [], 'phase': [], 'pv': [], 'mu': []}

    for _, row in data.iterrows():
        a, p, pv, mu = return_amp_phase_pv(np.array(row)[feature_pos], Ts, T, N)
        results['amp'].append(a)
        results['phase'].append(p)
        results['pv'].append(pv)
        results['mu'].append(mu)

    # Convert lists to numpy arrays
    for key in results:
        results[key] = np.array(results[key])

    return results


#### Helper functions

In [ ]:
# Apply the function to both intron and exon data
Ts = 2.0
T = 24
N = 48

results_intron = process_data(dat, 'Intron', Ts, T, N)
results_exon = process_data(dat, 'Exon', Ts, T, N)

# Select rhythmic genes based on p-value for example
ii=np.logical_and(results_intron['pv'] < 0.05,results_exon['pv'] < 0.05)
#... 
#...

#Perform the different plots


### Question 2
1. Related to Exercise 1, study the phase relationship between the pre-mRNA and mRNA for genes that exhibit rhythmic behavior at both the pre-mRNA and mRNA levels. Use appropriate cutoffs in your analysis to select a subset of genes that are rhythmic at both the pre-mRNA and mRNA levels. Aim to include several hundred genes for a robust analysis. 

2. Show representative profiles of genes with short or large delays.  

3. Investigate (e.g. use a scatter plot) the relation between the phase difference between pre-mRNA and mRNA and the amplitude ratio (log2 fold change) of pre-mRNA and mRNA.
    * Discuss the plot? E.g. Mark some outliers with their gene names. *Hint*: Identify genes that show exceptionally large/small phase differences or amplitude ratios compared to the general trend. 

